# 01_Synthetic_data_generator

## Section 1: Introduction & Goals
---
 This notebook simulates realistic insurance data for brokers/agencies.
**Goals**:
- Generate clients, agents, policies, claims, and sales data.
- Include extra features useful for segmentation, risk analysis, and predictive analytics.
- Inject controlled noise to simulate real-world cleaning tasks.

---

## Section 2: Import Libraries


In [ ]:
import pandas as pd
import numpy as np
from faker import Faker
import random
from datetime import datetime, timedelta
from faker.providers import BaseProvider
import os
from dateutil.relativedelta import relativedelta


In [ ]:
# Create output
os.makedirs("./data/01_raw_data", exist_ok= True)


## Section 3: Define parameters

In [ ]:
NUM_CLIENTS = 2000
NUM_AGENTS = 30
SIMULATION_YEARS = 3
START_DATE = datetime.now() - timedelta(days=SIMULATION_YEARS*365)
END_DATE = datetime.now()

policy_types = ['Life', 'Health', 'Motor', 'Property']
channels = ['Branch','Phone','Agent Visit','Online','Referral']
regions = ['Nakuru CBD','Gilgil','Naivasha','Molo','Rongai']
region_weights = [0.5, 0.15, 0.15, 0.1, 0.1]
lapse_reasons = ['Non-payment', 'Better offer elsewhere', 'Financial hardship', 'No longer needed', 'Dissatisfaction']
fake = Faker()

In [ ]:
class KenyanNameProvider(BaseProvider):
    first_names = ['John','Brian','Kevin','Jackson','Moses','Boniface','Phillip','Fredrick','Erik','Victor','Michael','Peter','Philemon','Anthony','David','Collins','Robert','Grace','Mary','Jane','Susan','Joyce','Elizabeth','Margaret','Catherine','Rose','Agnes','Esther','Faith','Ruth','Lucy','Nancy']
    last_names = ['Muchiri','Kamau','Odhiambo','Mutiso','Kariuki','Njoroge','Omondi','Korir','Owino','Ochieng','Mwangi','Wekesa','Barasa','Kiplagat','Kosgey','Wanjiku','Nyong','Kiprotich','Cheruiyot','Mbugua']
    
    def kenyan_name(self):
        return f"{random.choice(self.first_names)} {random.choice(self.last_names)}"

fake.add_provider(KenyanNameProvider)

## Section 4: Generate Agents Table

In [ ]:
agents = []
for aid in range(1, NUM_AGENTS+1):
    name = fake.kenyan_name()
    hire_date = fake.date_between(start_date='-7y', end_date='-6m')
    region = random.choices(regions, weights=region_weights, k=1)[0]
    
    # Agent performance tier affects conversion and retention rates
    performance_tier = random.choices(['Low', 'Medium', 'High'], weights=[0.2, 0.6, 0.2])[0]
    
    agents.append([aid, name, hire_date, region, performance_tier])

df_agents = pd.DataFrame(agents, columns=['AgentID','Name','HireDate','Region','PerformanceTier'])

##  Section 5: Generate Clients Table

In [ ]:
clients = []
for cid in range(1, NUM_CLIENTS + 1):
    name = fake.kenyan_name()
    dob = fake.date_of_birth(minimum_age=20, maximum_age=75)
    age = (pd.Timestamp('today') - pd.to_datetime(dob)).days // 365
    gender = random.choice(['Male', 'Female'])
    location = random.choices(regions, weights=region_weights, k=1)[0]
    join_date = fake.date_between(start_date=START_DATE, end_date=END_DATE - timedelta(days=180))

    # Enhanced client segmentation
    tier = random.choices(['Bronze','Silver','Gold','Platinum'], weights=[0.4, 0.35, 0.2, 0.05])[0]
    
    # Lifetime value with more sophisticated logic
    tier_multipliers = {'Bronze': 1, 'Silver': 2, 'Gold': 4, 'Platinum': 8}
    base_ltv = random.uniform(50000, 150000) * tier_multipliers[tier]
    
    # Age and location adjustments
    if age >= 50: base_ltv *= 1.15
    elif age <= 25: base_ltv *= 0.9
    if location == 'Nakuru CBD': base_ltv *= 1.2
    
    # Tenure bonus
    years_with_company = (pd.Timestamp('today') - pd.to_datetime(join_date)).days / 365
    base_ltv *= (1 + 0.05 * years_with_company)
    
    # Risk profile affects renewal probability
    risk_profile = random.choices(['Low', 'Medium', 'High'], weights=[0.3, 0.5, 0.2])[0]
    
    clients.append([cid, name, dob, age, gender, location, join_date, tier, round(base_ltv, 2), risk_profile])

df_clients = pd.DataFrame(clients, columns=['ClientID','Name','DOB','Age','Gender','Location','JoinDate','Tier','LifetimeValue','RiskProfile'])



## Section 6: Generate Policies Table

In [ ]:
policies = []
policy_id = 1
client_policies = {}

business_type_choices = ['Individual', 'SME', 'Corporate', 'Micro']
business_type_weights = [0.7, 0.2, 0.08, 0.02]

for client in df_clients.itertuples():
    client_id = client.ClientID
    client_join_date = pd.to_datetime(client.JoinDate)
    client_tier = client.Tier
    client_risk = client.RiskProfile

    months_since_join = (END_DATE - client_join_date).days // 30
    max_renewals = min(months_since_join // 12, SIMULATION_YEARS)

    num_policy_types = random.choices([1, 2, 3], weights=[0.6, 0.3, 0.1])[0]
    client_policy_types = random.sample(policy_types, num_policy_types)

    client_policies[client_id] = []

    for policy_type in client_policy_types:
        agent_id = random.randint(1, NUM_AGENTS)
        agent_performance = df_agents.loc[df_agents['AgentID']==agent_id, 'PerformanceTier'].values[0]

        current_start = client_join_date + timedelta(days=random.randint(0, 180))
        current_end = current_start + relativedelta(years=1)

        # Assign business type
        business_type = random.choices(business_type_choices, weights=business_type_weights)[0]

        # Premium logic with business type influence
        base_premiums = {'Motor': 20000, 'Health': 35000, 'Life': 80000, 'Property': 200000}
        premium = base_premiums[policy_type] * random.uniform(0.7, 1.5)

                # Coverage Type logic (only applies to Motor policies)
      # Coverage Type logic (only applies to Motor policies)
        if policy_type == 'Motor':
            coverage_type = 'TPO' if random.random() < 0.7 else 'Comprehensive'

            # Add vehicle details
            vehicle_age = random.randint(1, 15)  # years
            usage_type = random.choices(['Private', 'Commercial'], weights=[0.7, 0.3])[0]

            # Adjust premium for TPO (cheaper)
            if coverage_type == 'TPO':
                premium *= 0.3

            # Adjust premium based on vehicle age and usage
            if usage_type == 'Commercial':
                premium *= 1.4  # commercial is costlier
            if vehicle_age > 10:
                premium *= 0.8  # older vehicles cheaper
        else:
            coverage_type = None
            vehicle_age = None
            usage_type = None

        # Tier adjustments
        tier_multipliers = {'Bronze': 0.8, 'Silver': 1.0, 'Gold': 1.3, 'Platinum': 1.8}
        premium *= tier_multipliers[client_tier]

        # Business type adjustments
        if business_type == 'SME':
            premium *= 1.5
        elif business_type == 'Corporate':
            premium *= 3
        elif business_type == 'Micro':
            premium *= 0.3

        premium = round(premium, 2)

        # Risk score
        base_risk = {'Life':0.2,'Health':0.25,'Motor':0.3,'Property':0.15}[policy_type]
        risk_multiplier = {'Low': 0.8, 'Medium': 1.0, 'High': 1.3}[client_risk]
        risk_score = min(1.0, max(0.0, base_risk * risk_multiplier * random.uniform(0.8, 1.2)))

        channel = random.choices(channels, weights=[0.25,0.2,0.35,0.1,0.1])[0]

        status = 'Active' if current_end > datetime.now() else random.choices(['Active','Lapsed'], weights=[0.8,0.2])[0]

        # Compute a payment propensity score (0..1)
        base = 0.9  # baseline chance of on-time
        tier_penalty = {'Bronze': 0.9, 'Silver': 1.0, 'Gold': 1.05, 'Platinum': 1.1}[client_tier]
        risk_penalty = {'Low': 1.05, 'Medium': 1.0, 'High': 0.8}[client_risk]

        propensity = base * tier_penalty * risk_penalty
        propensity = min(0.99, max(0.01, propensity))  # clamp for safety

        # Convert to category probabilities
        p_on = propensity
        p_def = (1 - propensity) * 0.3
        p_late = 1 - p_on - p_def

        payment_status = random.choices(['On-time', 'Late', 'Default'], weights=[p_on, p_late, p_def])[0]


        # Append initial policy
        policies.append([
            policy_id, client_id, agent_id, policy_type, current_start, current_end, 
            premium, status, channel, payment_status, round(risk_score,3),
            None, 0, business_type, 'Individual',coverage_type, vehicle_age, usage_type
        ])

        original_policy_id = policy_id
        client_policies[client_id].append(policy_id)
        policy_id += 1

        # Generate renewals
        for renewal_num in range(1, max_renewals + 1):
            current_start = current_end
            current_end = current_start + relativedelta(years=1)

            if current_start >= END_DATE:
                break

            base_renewal_prob = {'Bronze': 0.6, 'Silver': 0.75, 'Gold': 0.85, 'Platinum': 0.9}[client_tier]
            risk_adjustment = {'Low': 1.1, 'Medium': 1.0, 'High': 0.8}[client_risk]
            agent_adjustment = {'Low': 0.9, 'Medium': 1.0, 'High': 1.1}[agent_performance]

            renewal_prob = base_renewal_prob * risk_adjustment * agent_adjustment

            if random.random() > renewal_prob:
                status = 'Lapsed'
                break
            else:
                # Increase premium slightly on renewal
                premium *= random.uniform(1.02, 1.08)
                premium = round(premium, 2)

                if random.random() < 0.1:
                    agent_id = random.randint(1, NUM_AGENTS)

                status = 'Active' if current_end > datetime.now() else random.choices(['Active','Lapsed'], weights=[0.85,0.15])[0]

                # Recompute payment status
                payment_status = random.choices(['On-time','Late','Default'], weights=[0.85,0.12,0.03])[0]

                policies.append([
                    policy_id, client_id, agent_id, policy_type, current_start, current_end, 
                    premium, status, channel, payment_status, round(risk_score,3),
                    original_policy_id, renewal_num, business_type, 'Renewal',
                    coverage_type, vehicle_age, usage_type  # carry forward same values
                ])

                client_policies[client_id].append(policy_id)
                policy_id += 1

# Build DataFrame
df_policies = pd.DataFrame(policies, columns=[
    'PolicyID','ClientID','AgentID','PolicyType','StartDate','EndDate','PremiumAmount',
    'Status','Channel','PaymentStatus','RiskScore','OriginalPolicyID','RenewalNumber','BusinessType','TransactionType','CoverageType','VehicleAge','UsageType'
])


## Section 7: Generate Sales Table

In [ ]:
import pandas as pd
import random

# Build sales
sales = []
sale_id = 1

for policy in df_policies.itertuples():
    agent_id = policy.AgentID
    sale_date = policy.StartDate
    premium = policy.PremiumAmount
    policy_type = policy.PolicyType
    transaction_type = policy.TransactionType  # Individual or Renewal
    business_type = policy.BusinessType
    
    # Commission rates vary by product
    base_commission_rates = {'Motor': 0.15, 'Health': 0.12, 'Life': 0.20, 'Property': 0.10}
    commission_rate = base_commission_rates[policy_type]
    
    # Lower commission for renewals
    if transaction_type == 'Renewal':
        commission_rate *= 0.7
    
    commission = round(premium * commission_rate, 2)
    paid_flag = random.choices([1, 0], weights=[0.92, 0.08])[0]
    
    sales.append([
        sale_id, policy.PolicyID, agent_id, commission, sale_date, paid_flag,
        policy_type, transaction_type, business_type, round(commission_rate, 4)
    ])
    sale_id += 1

# Create DataFrame
df_sales = pd.DataFrame(sales, columns=[
    'SaleID','PolicyID','AgentID','CommissionAmount','SaleDate','CommissionPaidFlag',
    'PolicyType','TransactionType','BusinessType','CommissionRate'
])


## Section 8: Claims Table

In [ ]:
claims = []
claim_id = 1

claim_types = {
    'Motor': ['Accident', 'Theft', 'Fire'],
    'Health': ['Hospitalization', 'Surgery', 'Maternity'],
    'Life': ['Death', 'Disability'],
    'Property': ['Fire', 'Flood', 'Theft']
}

for policy in df_policies.itertuples():
    # Fewer claims on lapsed policies
    if policy.Status == 'Lapsed' and random.random() < 0.7:
        continue

    # Base probability + risk adjustment
    base_claim_rates = {'Motor': 0.18, 'Health': 0.22, 'Life': 0.05, 'Property': 0.08}
    claim_probability = base_claim_rates[policy.PolicyType] * (1 + min(policy.RiskScore, 0.5))

    # Seasonal adjustment BEFORE probability check
    if policy.PolicyType == 'Motor':
        current_month = pd.to_datetime(policy.StartDate).month
        if current_month in [4, 5, 10, 11]:  # Rainy months in Kenya
            claim_probability *= 1.4

    # Decide if claim occurs
    if random.random() < claim_probability:
        policy_start = pd.to_datetime(policy.StartDate)
        policy_end = min(pd.to_datetime(policy.EndDate), END_DATE)
        claim_date = fake.date_between(start_date=policy_start.date(), end_date=policy_end.date())

        # Claim type
        claim_type = random.choice(claim_types[policy.PolicyType])

        # Claim amount logic
        policy_premium = policy.PremiumAmount
        if policy.PolicyType == 'Life':
            claim_amount = round(random.uniform(policy_premium * 5, policy_premium * 20), 2)
        elif policy.PolicyType == 'Health':
            claim_amount = round(random.uniform(policy_premium * 0.1, policy_premium * 2), 2)
        else:
            claim_amount = round(random.uniform(policy_premium * 0.2, policy_premium * 3), 2)

        # Fraud flag and status
        fraud_flag = random.choices([0, 1], weights=[0.97, 0.03])[0]
        if fraud_flag:
            claim_status = random.choices(['Approved', 'Rejected', 'Under Investigation'], weights=[0.2, 0.4, 0.4])[0]
        else:
            claim_status = random.choices(['Approved', 'Rejected', 'Pending'], weights=[0.75, 0.10, 0.15])[0]

        claims.append([
            claim_id, policy.PolicyID, claim_date, claim_amount, claim_status, fraud_flag, claim_type
        ])
        claim_id += 1

df_claims = pd.DataFrame(claims, columns=[
    'ClaimID', 'PolicyID', 'ClaimDate', 'ClaimAmount', 'ClaimStatus', 'FraudFlag', 'ClaimType'
])


## 8. DASHBOARD-READY SUMMARY TABLES
---

## Monthly metrics for time series analysis

In [ ]:

monthly_metrics = []
current_date = START_DATE.replace(day=1)

while current_date <= END_DATE:
    month_end = (current_date + relativedelta(months=1)) - timedelta(days=1)
    
    # Policies starting this month
    month_policies = df_policies[
        (pd.to_datetime(df_policies['StartDate']) >= current_date) & 
        (pd.to_datetime(df_policies['StartDate']) <= month_end)
    ]
    
    # Claims this month
    month_claims = df_claims[
        (pd.to_datetime(df_claims['ClaimDate']) >= current_date) & 
        (pd.to_datetime(df_claims['ClaimDate']) <= month_end)
    ].merge(df_policies[['PolicyID','PolicyType','CoverageType']], on='PolicyID', how='left')
    
    for policy_type in policy_types:
        type_policies = month_policies[month_policies['PolicyType'] == policy_type]
        type_claims = month_claims[month_claims['PolicyType'] == policy_type]
        
        # TPO vs Comprehensive split
        tpo_count = len(type_policies[type_policies['CoverageType'] == 'TPO'])
        comp_count = len(type_policies[type_policies['CoverageType'] == 'Comprehensive'])
        
        monthly_metrics.append([
            current_date.strftime('%Y-%m'),
            policy_type,
            len(type_policies),  # Policies Sold
            round(type_policies['PremiumAmount'].sum(), 2) if len(type_policies) > 0 else 0,
            len(type_policies[type_policies['TransactionType'] == 'Individual']),  # Individual (New)
            len(type_policies[type_policies['TransactionType'] == 'Renewal']),  # Renewals
            tpo_count,
            comp_count,
            len(type_claims),  # Claims count
            round(type_claims['ClaimAmount'].sum(), 2) if len(type_claims) > 0 else 0,
            round(type_policies['RiskScore'].mean(), 3) if len(type_policies) > 0 else 0
        ])
    
    current_date += relativedelta(months=1)

df_monthly_metrics = pd.DataFrame(monthly_metrics, columns=[
    'Month','PolicyType','PoliciesSold','PremiumRevenue','NewBusiness',
    'Renewals','TPO_Count','Comprehensive_Count','ClaimsCount','ClaimsAmount','AvgRiskScore'
])


## Agent Performance Summary

In [ ]:

agent_performance = []
for agent in df_agents.itertuples():
    agent_policies = df_policies[df_policies['AgentID'] == agent.AgentID]
    agent_sales = df_sales[df_sales['AgentID'] == agent.AgentID]
    
    total_policies = len(agent_policies)
    total_premium = agent_policies['PremiumAmount'].sum()
    total_commission = agent_sales['CommissionAmount'].sum()
    
    new_business = len(agent_policies[agent_policies['TransactionType'] == 'Individual'])
    renewals = len(agent_policies[agent_policies['TransactionType'] == 'Renewal'])
    
    tpo_sales = len(agent_policies[agent_policies['CoverageType'] == 'TPO'])
    comp_sales = len(agent_policies[agent_policies['CoverageType'] == 'Comprehensive'])
    
    avg_premium = total_premium / total_policies if total_policies > 0 else 0
    commission_ratio = (total_commission / total_premium) if total_premium > 0 else 0
    
    # Retention rate (renewals / original policies by that agent)
    original_policies = agent_policies[agent_policies['RenewalNumber'] == 0]
    retention_rate = renewals / max(len(original_policies), 1)
    
    agent_performance.append([
        agent.AgentID, agent.Name, agent.Region, agent.PerformanceTier,
        total_policies, round(total_premium, 2), round(total_commission, 2),
        round(avg_premium, 2), round(commission_ratio, 3),
        new_business, renewals, retention_rate, tpo_sales, comp_sales
    ])

df_agent_performance = pd.DataFrame(agent_performance, columns=[
    'AgentID','Name','Region','PerformanceTier','TotalPolicies','TotalPremium',
    'TotalCommission','AvgPremium','CommissionRatio','NewBusiness','Renewals',
    'RetentionRate','TPO_Sales','Comprehensive_Sales'
])

## 9. COMMISSION PAYMENT DATES / AGING 

In [ ]:

# Only apply to paid commissions
paid_mask = df_sales['CommissionPaidFlag'] == 1

# Realistic delay distribution (in days)
delay_choices = np.array([7, 14, 21, 30, 45, 60])
delay_probs = np.array([0.30, 0.25, 0.20, 0.15, 0.07, 0.03])  # skewed toward faster payments

# Initialize PaymentDate column
df_sales['PaymentDate'] = pd.NaT

# Assign delays to paid commissions
num_paid = paid_mask.sum()
if num_paid > 0:
    assigned_delays = np.random.choice(delay_choices, size=num_paid, p=delay_probs)
    df_sales.loc[paid_mask, 'PaymentDate'] = pd.to_datetime(df_sales.loc[paid_mask, 'SaleDate']) + pd.to_timedelta(assigned_delays, unit='d')

# Compute how many days it took to pay
df_sales['DaysToPay'] = (pd.to_datetime(df_sales['PaymentDate']) - pd.to_datetime(df_sales['SaleDate'])).dt.days

# Compute aging for unpaid commissions (how long they have been outstanding)
df_sales['DaysOutstanding'] = np.where(
    df_sales['CommissionPaidFlag'] == 0,
    (END_DATE - pd.to_datetime(df_sales['SaleDate'])).dt.days,
    0
)

# Optional: check for negative or weird values (safety)
df_sales['DaysToPay'] = df_sales['DaysToPay'].clip(lower=0)
df_sales['DaysOutstanding'] = df_sales['DaysOutstanding'].clip(lower=0)


## 10. REALISTIC NOISE INJECTION (after predictions & payments are derived)

In [ ]:
rng = np.random.default_rng(123)

# Missing values
if len(df_clients) > 0:
    df_clients.loc[df_clients.sample(frac=0.02, random_state=1).index, 'DOB'] = pd.NaT
    df_clients.loc[df_clients.sample(frac=0.01, random_state=2).index, 'Gender'] = None
    df_clients.loc[df_clients.sample(frac=0.005, random_state=3).index, 'Location'] = None

if len(df_policies) > 0:
    df_policies.loc[df_policies.sample(frac=0.01, random_state=4).index, 'PremiumAmount'] = np.nan
    df_policies.loc[df_policies.sample(frac=0.005, random_state=5).index, 'Channel'] = None

# Duplicates (simulate entry duplication)
duplicate_clients = df_clients.sample(frac=0.005, random_state=6)
df_clients = pd.concat([df_clients, duplicate_clients], ignore_index=True)

duplicate_policies = df_policies.sample(frac=0.01, random_state=7)
df_policies = pd.concat([df_policies, duplicate_policies], ignore_index=True)

# Outliers / entry errors
if len(df_policies) > 0:
    outlier_idx = df_policies.sample(frac=0.005, random_state=8).index
    df_policies.loc[outlier_idx, 'PremiumAmount'] = df_policies.loc[outlier_idx, 'PremiumAmount'] * 10
    
    negative_idx = df_policies.sample(frac=0.002, random_state=9).index
    df_policies.loc[negative_idx, 'PremiumAmount'] = -df_policies.loc[negative_idx, 'PremiumAmount'].abs()

# Categorical typos / unknowns
if len(df_clients) > 0:
    df_clients.loc[df_clients.sample(frac=0.002, random_state=12).index, 'Location'] = 'Unknown'

if len(df_policies) > 0:
    df_policies.loc[df_policies.sample(frac=0.002, random_state=13).index, 'Status'] = 'Actiev'  # typo



In [ ]:
# Validation Check
print(f"Clients columns are f{df_clients.columns}")
print(f"Agents columns are f{df_agents.columns}")
print(f"Policies columns are f{df_policies.columns}")
print(f"Sales columns are f{df_sales.columns}")
print(f"Claims columns are f{df_claims.columns}")
print(f"Monthly_metrics columns are f{df_monthly_metrics.columns}")
print(f"Agent_performance columns are f{df_agent_performance.columns}")

In [ ]:
# Save each dataframe into the folder
df_clients.to_csv('../data/01_raw_data/clients.csv', index=False)
df_agents.to_csv('../data/01_raw_data/agents.csv', index=False)
df_policies.to_csv('../data/01_raw_data/policies.csv', index=False)
df_sales.to_csv('../data/01_raw_data/sales.csv', index=False)
df_claims.to_csv('../data/01_raw_data/claims.csv', index=False)
df_monthly_metrics.to_csv('../data/01_raw_data/monthly_metrics.csv', index=False)
df_agent_performance.to_csv('../data/01_raw_data/agent_performance.csv', index=False)